In [1]:
import os
import sys
from pathlib import Path
curr_path = str(Path(os.getcwd()).parent)
sys.path.append(curr_path)
from scripts.sa2_age_allocation import *
from scripts.constants import *
from scripts.load import *
from scripts.transform import *
from scripts.read import *
from scripts.misc_changes import *
from scripts.join import *
from scripts.plotting import *
import warnings
warnings.filterwarnings("ignore")
from pyspark.sql.functions import *
from pyspark.sql.column import *
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import DecisionTreeClassifier, LogisticRegression
from pyspark.ml.regression import DecisionTreeRegressor, LinearRegression, RandomForestRegressor, LinearRegressionModel
from pyspark.ml.evaluation import RegressionEvaluator
import geopandas as gpd
import pandas as pd 
import numpy as np
import math
import re
import random
import json
spark = create_spark()
PREFIX = "."
# from scripts.model_linreg import *

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/30 15:12:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/30 15:12:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
consumer_external = spark.read.parquet("../data/curated/consumer_external_join_age_allocated.parquet")
tbl_merchants = read_curated_tbl_merchant(spark, PREFIX)
merchant_fraud = spark.read.csv("../data/curated/merchant_fraud_probability.csv/", header=True)
transactions = spark.read.parquet("../data/raw/raw_transactions/")

In [ ]:
def transactions_drop_columns(df):
    # Drop Male and Female Age coluns
    gender_age_cols = [col for col in df.columns if "male" in col]
    gender_cols = [col for col in df.columns if f"{GENDER}_" in col]
    
    revenue_level_cols = [col for col in df.columns if REVENUE_LEVEL in col]
    revenue_level_cols = sorted(revenue_level_cols, key=lambda x:len(x))[1:]

    # Order cols
    order_year_cols = [col for col in df.columns if ORDER_YEAR in col]
    order_year_cols = sorted(order_year_cols, key=lambda x:len(x))[1:]

    order_year_cols = [col for col in df.columns if ORDER_YEAR in col]
    order_year_cols = sorted(order_year_cols, key=lambda x:len(x))[1:]

    order_year_cols = [col for col in df.columns if ORDER_YEAR in col]
    order_year_cols = sorted(order_year_cols, key=lambda x:len(x))[1:]

    order_year_cols = [col for col in df.columns if ORDER_YEAR in col]
    order_year_cols = sorted(order_year_cols, key=lambda x:len(x))[1:]

    order_month_cols = [col for col in df.columns if ORDER_MONTH in col]
    order_month_cols = sorted(order_month_cols, key=lambda x:len(x))[1:]

    order_dom_cols = [col for col in df.columns if ORDER_DAY_OF_MONTH in col]
    order_dom_cols = sorted(order_dom_cols, key=lambda x:len(x))[1:]

    order_dow_cols = [col for col in df.columns if ORDER_DAY_OF_WEEK in col]
    order_dow_cols = sorted(order_dow_cols, key=lambda x:len(x))[1:]

    state_cols = [col for col in df.columns if f"{STATE_NAME}_" in col]
    sa2_cols = [col for col in df.columns if SA2_NAME in col]

    earningsum_cols = sorted([col for col in df.columns if "earningsum" in col])[:-1]
    medianage_cols = sorted([col for col in df.columns if "median_age" in col])[:-1]
    earningmedian_cols = sorted([col for col in df.columns if "earningmedian" in col])[:-1]
    earningmean_cols = sorted([col for col in df.columns if "eariningmean" in col])[:-1]
    earners_cols = sorted([col for col in df.columns if "earners" in col])[:-1]
    industry_tag_cols = sorted([col for col in df.columns if "industry_tag" in col])[1:]
    additional_drop = gender_age_cols + earningmedian_cols + medianage_cols + earningsum_cols +\
        earningmean_cols + earners_cols + [RATIO, SA2_CODE_2016, NAME] + industry_tag_cols +\
            order_dom_cols + order_dow_cols + order_month_cols + order_year_cols + \
                gender_age_cols + revenue_level_cols + gender_cols + state_cols + sa2_cols\
    
    return tuple(additional_drop)

def consumer_external_drop_columns(df):
    # Drop Male and Female Age coluns
    gender_age_cols = [col for col in df.columns if "male" in col]
    gender_cols = [col for col in df.columns if f"{GENDER}_" in col]
    
    revenue_level_cols = [col for col in df.columns if REVENUE_LEVEL in col]
    revenue_level_cols = sorted(revenue_level_cols, key=lambda x:len(x))[1:]

    # Order cols
    order_year_cols = [col for col in df.columns if ORDER_YEAR in col]
    order_year_cols = sorted(order_year_cols, key=lambda x:len(x))[1:]

    order_year_cols = [col for col in df.columns if ORDER_YEAR in col]
    order_year_cols = sorted(order_year_cols, key=lambda x:len(x))[1:]

    order_year_cols = [col for col in df.columns if ORDER_YEAR in col]
    order_year_cols = sorted(order_year_cols, key=lambda x:len(x))[1:]

    order_year_cols = [col for col in df.columns if ORDER_YEAR in col]
    order_year_cols = sorted(order_year_cols, key=lambda x:len(x))[1:]

    order_month_cols = [col for col in df.columns if ORDER_MONTH in col]
    order_month_cols = sorted(order_month_cols, key=lambda x:len(x))[1:]

    order_dom_cols = [col for col in df.columns if ORDER_DAY_OF_MONTH in col]
    order_dom_cols = sorted(order_dom_cols, key=lambda x:len(x))[1:]

    order_dow_cols = [col for col in df.columns if ORDER_DAY_OF_WEEK in col]
    order_dow_cols = sorted(order_dow_cols, key=lambda x:len(x))[1:]

    state_cols = [col for col in df.columns if f"{STATE_NAME}_" in col]
    sa2_cols = [col for col in df.columns if SA2_NAME in col]

    earningsum_cols = sorted([col for col in df.columns if "earningsum" in col])
    medianage_cols = sorted([col for col in df.columns if "median_age" in col])
    earningmedian_cols = sorted([col for col in df.columns if "earningmedian" in col])
    earningmean_cols = sorted([col for col in df.columns if "eariningmean" in col])
    earners_cols = sorted([col for col in df.columns if "earners" in col])
    industry_tag_cols = sorted([col for col in df.columns if "industry_tag" in col])
    additional_drop = gender_age_cols + earningmedian_cols + medianage_cols + earningsum_cols +\
        earningmean_cols + earners_cols + [RATIO, SA2_CODE_2016, NAME] + industry_tag_cols +\
            order_dom_cols + order_dow_cols + order_month_cols + order_year_cols + \
                gender_age_cols + revenue_level_cols + gender_cols + state_cols + sa2_cols\
    
    return tuple(additional_drop)


In [ ]:
# merchants = 

In [ ]:
# tbl_merchants = spark.read.parquet("../data/curated/tbl_merchants.parquet/")

In [ ]:
# # file imports
# transactions = spark.read.parquet(PREFIX+RAW_TRANSACTIONS_PATH)
# transactions = round_dollar_values(transactions)
# transactions = transactions.join(consumer_external, on=USER_ID, how=INNER_JOIN)
# transactions = transactions.join(tbl_merchants, on=MERCHANT_ABN, how=INNER_JOIN)


In [ ]:
# print(transactions_1.count())
# print(transactions_2.count())
# print(transactions_3.count())


In [ ]:
# all_transactions = read_curated_transactions_all(spark, PREFIX)
# all_transactions.printSchema()

In [ ]:
# CURATED_TRANSACTIONS_ALL_PATH

In [ ]:
def prepare_train_test_files(consumer_external: DataFrame):
    JOIN_COLUMNS = [USER_ID, SA2_CODE, STATE, POSTCODE, GENDER]
    INDUSTRY_1 = "tent awning"
    INDUSTRY_2 = "gift card novelty souvenir"
    INDUSTRY_3 = "digital goods books movies music"
    industries = [INDUSTRY_1, INDUSTRY_2, INDUSTRY_3]
    order_cols = (col(ORDER_YEAR), col(ORDER_MONTH), col(ORDER_DAY_OF_MONTH))
    INDEXED_COL = "_indexed"
    cat_cols = [MERCHANT_ABN, SA2_CODE, STATE, USER_ID, GENDER, ALLOCATED_AGES, INDUSTRY_TAGS]

    print("IMPORT ALL TRANSACTION FILES")
    transactions_all_files = [f"{PREFIX}{CURATED_TRANSACTIONS_ALL_PATH}{fname}" for fname in os.listdir(PREFIX+CURATED_TRANSACTIONS_ALL_PATH)]

    # First 6 files are train, next 3 are test
    print("DEFINE TRAIN AND TEST FILES")
    train_data_files = transactions_all_files[0:9]
    # train_data_files = transactions_all_files[0:6]
    # validate_data_files = transactions_all_files[6:9]
    test_data_files = transactions_all_files[9:12]
    
    print("DROP IRRELEVANT COLUMNS FOR CONSUMER EXTERNAL")
    drop_cols = consumer_external_drop_columns(consumer_external)
    consumer_external = consumer_external.drop(*(drop_cols))

    # train_data_files
    print("READ TRAIN/TEST FILES")
    train_df = spark.read.parquet(*train_data_files)
    # validate_df = spark.read.parquet(*validate_data_files)
    test_df = spark.read.parquet(*test_data_files)

    # Now save the model
    print("PERFORM CLEANING/TRANSFORMATIONS FOR TRAIN")
    drop_cols = transactions_drop_columns(train_df)
    test_path = "./test_files/model_save_test/"
    train_df = train_df.drop(*drop_cols)
    train_df = cast_data_type(train_df)
    train_df = encode_revenue_level(train_df)
    train_df = train_df.withColumns({
        USER_ID: col(USER_ID).cast(StringType()),
        MERCHANT_ABN: col(MERCHANT_ABN).cast(StringType())
    })
    
    # print("PERFORM CLEANING/TRANSFORMATIONS FOR VALIDATE")
    # validate_df = validate_df.drop(*drop_cols)
    # validate_df = cast_data_type(validate_df)
    # validate_df = encode_revenue_level(validate_df)
    # validate_df = validate_df.withColumns({
    #     USER_ID: col(USER_ID).cast(StringType()),
    #     MERCHANT_ABN: col(MERCHANT_ABN).cast(StringType())
    # })
    
    print("PERFORM CLEANING/TRANSFORMATIONS FOR TEST")
    test_df = test_df.drop(*drop_cols)
    test_df = cast_data_type(test_df)
    test_df = encode_revenue_level(test_df)
    test_df = test_df.withColumns({
        USER_ID: col(USER_ID).cast(StringType()),
        MERCHANT_ABN: col(MERCHANT_ABN).cast(StringType())
    })
    
    print("PERFORM JOINING TO CONSUMER ROWS")
    train_transactions = train_df.join(consumer_external, on=JOIN_COLUMNS, how=INNER_JOIN)
    # validate_transactions = validate_df.join(consumer_external, on=JOIN_COLUMNS, how=INNER_JOIN)
    test_transactions = test_df.join(consumer_external, on=JOIN_COLUMNS, how=INNER_JOIN)
    
    print("PERFORM DELETION OF SPECIFIC COLUMNS")
    train_transactions = train_transactions.drop(ORDER_ID, POSTCODE, ORDER_DAY_OF_WEEK)
    # validate_transactions = validate_transactions.drop(ORDER_ID, POSTCODE, ORDER_DAY_OF_WEEK)
    test_transactions = test_transactions.drop(ORDER_ID, POSTCODE, ORDER_DAY_OF_WEEK)
    
    print("PERFORM FILTERING OF TOP 3 INDUSTRIES")
    train_transactions = train_transactions.where(col(INDUSTRY_TAGS).isin(industries)).orderBy(*order_cols)
    # validate_transactions = validate_transactions.where(col(INDUSTRY_TAGS).isin(industries)).orderBy(*order_cols)
    test_transactions = test_transactions.where(col(INDUSTRY_TAGS).isin(industries)).orderBy(*order_cols)
    
    df_columns = train_transactions.columns
    
    print("PERFORM STRING INDEXING")
    for column in cat_cols:
        col_indexer = StringIndexer(inputCol=column, outputCol=column+INDEXED_COL, handleInvalid="keep")
        df_columns.append(column+INDEXED_COL)
        fitted_indexer = col_indexer.fit(train_transactions)
        train_transactions = fitted_indexer.transform(train_transactions)
        # validate_transactions = fitted_indexer.transform(validate_transactions)
        test_transactions = fitted_indexer.transform(test_transactions)

    df_columns = [col for col in df_columns if col not in cat_cols]
    df_columns = [col for col in df_columns if col != DOLLAR_VALUE]
    print(df_columns)
    
    # print("PERFORM VECTOR ASSEMBLING")
    # assembler = VectorAssembler(inputCols=df_columns, outputCol="features")
    # train_vec_transactions = assembler.transform(train_transactions)
    # # validate_vec_transactions = assembler.transform(validate_transactions)
    # test_vec_transactions = assembler.transform(test_transactions)

    # return train_vec_transactions, test_vec_transactions
    return
        

In [ ]:
def perform_hyperparams_tuning_lr(train, test):
    LEARNING_RATE = 0.01
    L1_PENALTY = 0.1
    NUM_ITERS = 100
    EPSILON = 0.000001
    mae_scores = []

    # Hyperparamters to tune for
    REG_PARAMS = [0.001, 0.01, 0.1, 1, 10, 100]
    ELASTIC_NET_PARAMS = [0.0, 0.25, 0.5, 0.75, 1.0]
    
    for reg in REG_PARAMS:
        for net in ELASTIC_NET_PARAMS:
            print(f"Reg: {reg}, Net: {net}")
            lr = LinearRegression(featuresCol="features", labelCol=DOLLAR_VALUE, 
                            predictionCol=f"predicted_{DOLLAR_VALUE}",
                            maxIter=100, regParam=reg, elasticNetParam=net)
            lr_model = lr.fit(train)
            predictions = lr_model.transform(test)
            
            evaluator = RegressionEvaluator(labelCol=DOLLAR_VALUE, predictionCol=f"predicted_{DOLLAR_VALUE}", metricName="mae")
            score = evaluator.evaluate(predictions)
            mae_scores.append(score)
            print(f"Score: {score}")
    
    return mae_scores

In [ ]:
def run_linear_regression(train, test, params: dict()):

    print("BEGIN TRAINING MODEL")
    lr = LinearRegression(featuresCol="features", labelCol=DOLLAR_VALUE, 
                            predictionCol=f"predicted_{DOLLAR_VALUE}",
                            regParam=params["reg"], elasticNetParam=params["net"])
    lr_model = lr.fit(train)
    print("PRODUCE PREDICTIONS")
    predictions = lr_model.transform(test)

    return lr_model, predictions

In [ ]:
train_vec, test_vec = prepare_train_test_files(consumer_external)
# prepare_train_test_files(consumer_external)

In [ ]:
mae_scores = perform_hyperparams_tuning_lr(train_vec, test_vec)

In [ ]:
params = {"reg": 100, "net": 0.25}

In [ ]:
lr_model, predictions = run_linear_regression(train_vec, test_vec, params)

In [ ]:
lr_model.coefficients

In [ ]:
evaluator = RegressionEvaluator(labelCol=DOLLAR_VALUE, predictionCol=f"predicted_{DOLLAR_VALUE}", metricName="r2")
score = evaluator.evaluate(predictions)
score

In [ ]:
PREDICTED_DOLLARS = "predicted_dollar_value"
RESIDUALS = "residuals"
pred_values = predictions.select(DOLLAR_VALUE, f"predicted_{DOLLAR_VALUE}")
pred_values = pred_values.withColumn(PREDICTED_DOLLARS, round(PREDICTED_DOLLARS, 2))
pred_values = pred_values.withColumn(RESIDUALS, col(DOLLAR_VALUE) - col(PREDICTED_DOLLARS))
pred_values.count()

In [ ]:
initial_values, sample_values = pred_values.randomSplit([0.995, 0.005])

In [ ]:
sample_values.count()

In [ ]:
residuals = list(sample_values.select(RESIDUALS).rdd.flatMap(lambda x: x).collect())
predicted_dollar = list(sample_values.select(PREDICTED_DOLLARS).rdd.flatMap(lambda x: x).collect())
actual_dollar = list(sample_values.select(DOLLAR_VALUE).rdd.flatMap(lambda x: x).collect())

In [ ]:
# sample_values.select(PREDICTED_DOLLARS).count()
predicted_dollar = list(sample_values.select(PREDICTED_DOLLARS).rdd.flatMap(lambda x: x).collect())
len(predicted_dollar)

In [ ]:
len(actual_dollar)

In [ ]:
# Calculate the standard deviation of residuals
residual_std = np.std(residuals)

# Calculate standardized residuals
standardized_residuals = residuals / residual_std

In [ ]:
# Create a scatter plot of residuals against predicted values
plt.figure(figsize=(8, 6))
plt.scatter(predicted_dollar, actual_dollar, alpha=0.5, c="#C76528")
plt.title("Predicted vs Actual Dollar Values")
plt.xlabel("Predicted values ($)")
plt.ylabel("Actual values ($)")
plt.savefig("../plots/predicted_vs_actual_dollars.png")
plt.show()


In [ ]:
# Create a scatter plot of residuals against predicted values
plt.figure(figsize=(8, 6))
plt.scatter(predicted_dollar, standardized_residuals, alpha=0.5, c="#C76528")
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Predicted Dollar Values vs. Residuals")
plt.xlabel("Predicted values ($)")
plt.ylabel("Standardised Residuals")
plt.savefig("../plots/predicted_vs_residuals.png")
plt.show()

### Linear Regression Model

In [ ]:
joined_transactions = joined_transactions.drop(ORDER_ID, POSTCODE, ORDER_DAY_OF_WEEK)
INDUSTRY_1 = "tent awning"
INDUSTRY_2 = "gift card novelty souvenir"
INDUSTRY_3 = "digital goods books movies music"
industries = [INDUSTRY_1, INDUSTRY_2, INDUSTRY_3]
INDEXED_COL = "_indexed"
cat_cols = [MERCHANT_ABN, SA2_CODE, STATE, USER_ID, GENDER, ALLOCATED_AGES, INDUSTRY_TAGS]

joined_transactions = joined_transactions.where(col(INDUSTRY_TAGS).isin(industries)).orderBy(col(ORDER_YEAR), col(ORDER_MONTH), col(ORDER_DAY_OF_MONTH))
# transactions_1 = joined_transactions.where(col(INDUSTRY_TAGS) == INDUSTRY_1).orderBy(col(ORDER_YEAR), col(ORDER_MONTH), col(ORDER_DAY_OF_MONTH))
# transactions_2 = joined_transactions.where(col(INDUSTRY_TAGS) == INDUSTRY_2).orderBy(col(ORDER_YEAR), col(ORDER_MONTH), col(ORDER_DAY_OF_MONTH))
# transactions_3 = joined_transactions.where(col(INDUSTRY_TAGS) == INDUSTRY_3).orderBy(col(ORDER_YEAR), col(ORDER_MONTH), col(ORDER_DAY_OF_MONTH))

In [ ]:
joined_transactions.show(5)

In [ ]:
order_cols = (col(ORDER_YEAR), col(ORDER_MONTH), col(ORDER_DAY_OF_MONTH))
joined_transactions.orderBy(*order_cols)

In [ ]:
df_columns = joined_transactions.columns
for column in cat_cols:
    col_indexer = StringIndexer(inputCol=column, outputCol=column+INDEXED_COL)
    df_columns.append(column+INDEXED_COL)
    joined_transactions = col_indexer.fit(joined_transactions).transform(joined_transactions)

df_columns = [col for col in df_columns if col not in cat_cols]

assembler = VectorAssembler(inputCols=df_columns, outputCol="features")
vector_transactions = assembler.transform(joined_transactions)

lr = LinearRegression(featuresCol="features", labelCol=DOLLAR_VALUE, 
                          predictionCol=f"predicted_{DOLLAR_VALUE}")
lr_model = lr.fit(vector_transactions)

mae_evaluator = RegressionEvaluator(labelCol=DOLLAR_VALUE, predictionCol=f"predicted_{DOLLAR_VALUE}", metricName="mae")
r2_evaluator = RegressionEvaluator(labelCol=DOLLAR_VALUE, predictionCol=f"predicted_{DOLLAR_VALUE}", metricName="r2")
mae_score = mae_evaluator.evaluate(predictions)
r2_score = r2_evaluator.evaluate

In [ ]:
lr = LinearRegression(featuresCol="features", labelCol=DOLLAR_VALUE, 
                          predictionCol=f"predicted_{DOLLAR_VALUE}")
lr_model = lr.fit(vector_transactions)

mae_evaluator = RegressionEvaluator(labelCol=DOLLAR_VALUE, predictionCol=f"predicted_{DOLLAR_VALUE}", metricName="mae")
r2_evaluator = RegressionEvaluator(labelCol=DOLLAR_VALUE, predictionCol=f"predicted_{DOLLAR_VALUE}", metricName="r2")
mae_score = mae_evaluator.evaluate(predictions)
r2_score = r2_evaluator.evaluate

In [ ]:
# fit_linreg(train_df=all_transactions, save_path=SAVE_PATH, target_col=DOLLAR_VALUE, drop_cols=drop_cols, prefix=PREFIX)

In [ ]:
# all_transactions.printSchema()

In [ ]:
# test_model_linreg()

### Age Distribution Analysis

In [ ]:
age_allocation = spark.read.parquet("../data/curated/consumer_external_join_age_allocated.parquet")
male_consumers = age_allocation.where(col(GENDER) == MALE) 
female_consumers = age_allocation.where(col(GENDER) == FEMALE) 

In [ ]:
male_count = male_consumers.groupBy(SA2_CODE, ALLOCATED_AGES).count()
female_count = female_consumers.groupBy(SA2_CODE, ALLOCATED_AGES).count()